# Data Cleaned

In [151]:
import numpy as np
import pandas as pd

In [152]:
df_vacc = pd.read_csv('child_vaccination_data.csv')
num_rows_original = df_vacc.shape[0]
df_vacc.shape

(118423, 10)

In [153]:
df_vacc.head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
0,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,>400% FPL,64.2,55.3 to 73.1,189.0
1,Hep A,≥1 Dose,States/Local Areas,Tennessee,2016-2019,Race and Ethnicity,Hispanic,86.4,76.8 to 95.9,187.0
2,PCV,≥4 Doses,States/Local Areas,Tennessee,2016-2019,Insurance Coverage,Private Insurance Only,89.3,85.4 to 93.2,612.0
3,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,133% to <400% FPL,51.4,44.0 to 58.8,342.0
4,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,<133% FPL,39.0,31.9 to 46.1,298.0


In [154]:
# pivot 'Dimension Type' into new columns (with 'Dimension' containing values for new columns)
df_vacc = pd.pivot(df_vacc, 
                   index=['Vaccine', 'Dose', 'Geography Type', 'Geography', 'Birth Year/Birth Cohort', \
                          'Dimension', 'Estimate (%)', '95% CI (%)', 'Sample Size'], # identifier for each row
                   columns=['Dimension Type'], # column containing new column names
                   values='Dimension')         # values for new columns

df_vacc.reset_index(inplace=True) 
df_vacc.rename_axis(None, axis=1, inplace=True) # remove the named index

In [155]:
# drop 'Dimension' column as it is no longer needed after pivot
df_vacc.drop(columns='Dimension', inplace = True)

In [156]:
# split 'Birth Year/Birth Cohort' column into two columns by looking for hyphen in values
df_vacc['Birth Year'] = np.where(df_vacc['Birth Year/Birth Cohort']\
                                 .str.contains('-'), np.nan, df_vacc['Birth Year/Birth Cohort'])
df_vacc['Birth Cohort'] = np.where(df_vacc['Birth Year/Birth Cohort']\
                                   .str.contains('-'), df_vacc['Birth Year/Birth Cohort'], np.nan)

In [157]:
# drop 'Birth Year/Birth Cohort' column as it is no longer needed after split
df_vacc.drop(columns='Birth Year/Birth Cohort', inplace = True)

In [158]:
# reorder columns
df_vacc = df_vacc[['Vaccine', 'Dose', 'Geography Type', 'Geography', 'Birth Year', 'Birth Cohort', \
                   'Estimate (%)', '95% CI (%)', 'Overall', 'Age', 'Race and Ethnicity', 'Poverty', \
                    'Insurance Coverage', 'Urbanicity', 'Sample Size']]

In [159]:
# check for NA in Estimate (%) which will be column needed in all data visualizations in app
df_vacc['Estimate (%)'].isna().sum()

58

In [160]:
# drop rows with NA in Estimate (%) which will be column needed in all data visualizations in app
df_vacc.dropna(subset=['Estimate (%)'], inplace=True)

num_rows_current = df_vacc.shape[0]

print(num_rows_original - num_rows_current, "rows dropped")

58 rows dropped


In [161]:
# fill NA values for Dose for certain vaccines (based on CDC documentation of what these represent)
df_vacc['Dose'] = np.where(df_vacc['Vaccine'] == 'Combined 7 Series', 'Full Series', df_vacc['Dose'])
df_vacc['Dose'] = np.where(df_vacc['Vaccine'] == 'Influenza', '≥2 Doses at least 24 days apart', df_vacc['Dose'])
df_vacc['Dose'] = np.where(df_vacc['Vaccine'] == 'Rotavirus', 'Full Series', df_vacc['Dose'])

In [162]:
# update values for HHS Regions to actually include 'HHS' in value to be more clear what Region means
df_vacc['Geography'] = np.where(df_vacc['Geography'].str.contains('Region'), \
                                df_vacc['Geography'].str.replace('Region', 'HHS Region'), df_vacc['Geography'])

In [163]:
# rename several columns to be more clear for use in app
df_vacc.rename(columns={'Geography': 'Geographic Area', 'Poverty': 'Poverty Level', \
                        'Insurance Coverage': 'Health Insurance Coverage'}, inplace=True)

In [164]:
# split 95% CI column into upper and lower bounds for possible use in visualizations
df_vacc['95% CI Lower (%)'] = df_vacc['95% CI (%)'].str.split(' to ').str[0]
df_vacc['95% CI Upper (%)'] = df_vacc['95% CI (%)'].str.split(' to ').str[1]

In [165]:
# export set of unique values for Geographic Area which will be manipulated to create preferred sort order for filter in app
# Preferred Order = US first, then HHS Regions by number, then States alphabetically with City/County ordered after their state
geo_areas = df_vacc['Geographic Area'].unique()
df_geo_areas = pd.DataFrame(geo_areas)
df_geo_areas.to_csv('geo_areas.csv', index=False)

In [150]:
df_vacc.sample(10)

,Vaccine,Dose,Geography Type,Geographic Area,Birth Year,Birth Cohort,Estimate (%),95% CI (%),Overall,Age,Race and Ethnicity,Poverty Level,Health Insurance Coverage,Urbanicity,Sample Size,95% CI Lower (%),95% CI Upper (%),CI Mean
99755,Polio,≥2 Doses,HHS Regions/National,HHS Region 1,2020,NaN,93.8,90.2 to 96.1,NaN,7 Months,NaN,NaN,NaN,NaN,946.0,90.2,96.1,93.15
28422,Hep A,≥2 Doses,States/Local Areas,Nebraska,NaN,2017-2018,59.7,54.6 to 64.9,NaN,24 Months,NaN,NaN,NaN,NaN,545.0,54.6,64.9,59.75
4025,Combined 7 Series,Full Series,States/Local Areas,West Virginia,2019,NaN,68.0,58.2 to 77.4,NaN,35 Months,NaN,NaN,NaN,NaN,201.0,58.2,77.4,67.80
114515,Varicella,≥1 Dose,States/Local Areas,Louisiana,2014,NaN,59.2,50.8 to 67.0,NaN,13 Months,NaN,NaN,NaN,NaN,261.0,50.8,67.0,58.90
25525,Hep A,≥2 Doses,HHS Regions/National,HHS Region 3,2018,NaN,78.9,74.8 to 82.7,NaN,35 Months,NaN,NaN,NaN,NaN,2888.0,74.8,82.7,78.75
64567,Hib,≥2 Doses,States/Local Areas,Oklahoma,NaN,2013-2014,71.8,68.8 to 74.6,NaN,5 Months,NaN,NaN,NaN,NaN,425.0,68.8,74.6,71.70
67519,Hib,≥3 Doses,States/Local Areas,Guam,NaN,2013-2014,72.6,67.6 to 77.0,NaN,13 Months,NaN,NaN,NaN,NaN,293.0,67.6,77.0,72.30
72317,Influenza,≥2 Doses at least 24 days apart,States/Local Areas,Alabama,NaN,2016-2019,37.8,31.1 to 44.5,NaN,NaN,NaN,133% to <400% FPL,NaN,NaN,395.0,31.1,44.5,37.80
28336,Hep A,≥2 Doses,States/Local Areas,NY-Rest of state,NaN,2016-2019,73.2,60.6 to 85.8,NaN,NaN,NaN,NaN,NaN,Living In a MSA Principal City,201.0,60.6,85.8,73.20
47243,Hep B,≥3 Doses,States/Local Areas,Rhode Island,NaN,2014-2017,95.6,93.2 to 98.0,NaN,NaN,NaN,NaN,Any Medicaid,NaN,361.0,93.2,98.0,95.60


In [166]:
# saved cleaned data to csv
df_vacc.to_csv('child_vaccination_data_cleaned.csv', index=False)